# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [7]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation']


In [8]:
import pandas as pd
import config as cfg
from tqdm import tqdm
import swifter
from pshmodule.utils import filemanager as fm

In [9]:
df = fm.load(cfg.opendict)

extension : .xlsx
Loaded 88508 records from drive/MyDrive/MemeProject/data/row_data/opendict.xlsx


In [10]:
df.rename(columns={'어휘':'vocab', '품사':'pos', '의미 번호':'mean_no', '뜻풀이':'mean'}, inplace=True)

In [11]:
df.head(10)

,vocab,pos,mean_no,mean
0,가가대소-하다,동사,1,소리 내어 크게 웃다.
1,가가-하다,형용사,1,잘되었거나 착하다고 여길 만하다.
2,가각-하다,형용사,1,인정이 없고 모질다.
3,가감-되다,동사,1,더해지거나 빼지다. 또는 더해지거나 빼져서 알맞게 맞추어지다.
4,가감-하다,동사,1,더하거나 빼다. 또는 더하거나 빼서 알맞게 맞추다.
5,가감-하다,형용사,2,어떤 일정한 일을 능히 해낼 수 있다.
6,가강-하다,형용사,1,더욱 강력하고 의지가 굳다.
7,가개통-하다,동사,1,"길, 다리, 철로, 전화, 전신 따위를 임시로 이어 통하게 하다."
8,가거-하다,동사,1,벼슬살이에서 물러나 집 안에서 한가롭게 지내다.
9,가거-하다,동사,2,과년한 처녀가 시집가지 아니하고 집에 머물다.


# Extract UserDict

모든 문장은 마침표 기준으로 스플릿

1. (가-힣)다.<br>
2. (가-힣)게 (가-힣)다.<br>
3. (가-힣)거나 (가-힣)다.<br>
4. (가-힣)고 (가-힣)다.<br>
5. 피동사, 사동사가 들어가지 않은 문장에서 '' 안에 들어가는 말을 추출(아래 모든 예시)<br>
  A의 말<br>
  ~의 준말, ~를 속되게 이르는 말, ~의 비유, ~의 예사스러운 말<br>
  규범 표기는 ‘감쪽같다’이다.
  ‘내보내다’의 본말.
  ‘너덜거리다’보다 거센 느낌을 준다.<br>
6. 본용언 + 연결어미 + 보조용언 -> 본용언을 추출<br>
버리다, 놓다 등 마지막 보조 용언 리스트를 하은 님이 주시면 앞 바로 앞 어절이 ~을, ~를로 끝나지 않는 것만 추출

##### 1. (가-힣)다

In [12]:
import re

In [13]:
def extract1(x):
  temp = x.split('.')
  temp = [i+'.' for i in temp if i != '']
  
  result = []
  for i in temp:
    m = re.search('[ㄱ-힣]+다.$', i.strip())
    if m:
      result.append(m.group().replace('.', ''))
  
  return ', '.join(result)

In [14]:
df['extract1'] = df['mean'].swifter.apply(extract1)

Pandas Apply:   0%|          | 0/88508 [00:00<?, ?it/s]

##### 2. (가-힣)게 (가-힣)다.

In [15]:
def extract2(x):
  temp = x.split('.')
  temp = [i+'.' for i in temp if i != '']
  
  result = []
  for i in temp:
    m = re.search('[ㄱ-힣]+게 [ㄱ-힣]+다.$', i.strip())
    if m:
      result.append(m.group().replace('.', ''))
  
  return ', '.join(result)

In [16]:
df['extract2'] = df['mean'].swifter.apply(extract2)

Pandas Apply:   0%|          | 0/88508 [00:00<?, ?it/s]

##### 3. (가-힣)거나 (가-힣)다.

In [17]:
def extract3(x):
  temp = x.split('.')
  temp = [i+'.' for i in temp if i != '']
  
  result = []
  for i in temp:
    m = re.search('[ㄱ-힣]+거나 [ㄱ-힣]+다.$', i.strip())
    if m:
      result.append(m.group().replace('.', ''))
  
  return ', '.join(result)

In [18]:
df['extract3'] = df['mean'].swifter.apply(extract3)

Pandas Apply:   0%|          | 0/88508 [00:00<?, ?it/s]

In [19]:
df.head()

,vocab,pos,mean_no,mean,extract1,extract2,extract3
0,가가대소-하다,동사,1,소리 내어 크게 웃다.,웃다,크게 웃다,
1,가가-하다,형용사,1,잘되었거나 착하다고 여길 만하다.,만하다,,
2,가각-하다,형용사,1,인정이 없고 모질다.,모질다,,
3,가감-되다,동사,1,더해지거나 빼지다. 또는 더해지거나 빼져서 알맞게 맞추어지다.,"빼지다, 맞추어지다",알맞게 맞추어지다,더해지거나 빼지다
4,가감-하다,동사,1,더하거나 빼다. 또는 더하거나 빼서 알맞게 맞추다.,"빼다, 맞추다",알맞게 맞추다,더하거나 빼다


##### 4. (가-힣)고 (가-힣)다.

In [20]:
def extract4(x):
  temp = x.split('.')
  temp = [i+'.' for i in temp if i != '']
  
  result = []
  for i in temp:
    m = re.search('[ㄱ-힣]+고 [ㄱ-힣]+다.$', i.strip())
    if m:
      result.append(m.group().replace('.', ''))
  
  return ', '.join(result)

In [21]:
df['extract4'] = df['mean'].swifter.apply(extract4)

Pandas Apply:   0%|          | 0/88508 [00:00<?, ?it/s]

##### 5. 피동사, 사동사가 들어가지 않은 문장에서 '' 안에 들어가는 말을 추출(아래 모든 예시)

In [114]:
def extract5(x):
  temp = x.split('.')
  temp = [i+'.' for i in temp if i != '']
  
  result = []
  for i in temp:
    if '피동사' not in i and '사동사' not in i:
      m = re.search("[ㄱ-힣]+’", i.strip())
      if m:
        result.append(m.group().replace('’', ''))
  return ', '.join(result)

In [116]:
df['extract5'] = df['mean'].swifter.apply(extract5)

Pandas Apply:   0%|          | 0/88508 [00:00<?, ?it/s]

In [119]:
check = df[df['mean'].str.contains("를 속되게 이르는 말")]

In [120]:
check.head()

,vocab,pos,mean_no,mean,extract1,extract2,extract3,extract4,extract5
2221,강짜-하다,동사,1,‘강샘하다’를 속되게 이르는 말.,,,,,강샘하다
5551,고꾸라-지다,동사,2,‘죽다’를 속되게 이르는 말.,,,,,죽다
5807,고뿔-들다,동사,1,‘임신하다’를 속되게 이르는 말.,,,,,임신하다
6485,공갈-치다,동사,1,‘공갈하다’를 속되게 이르는 말.,,,,,공갈하다
7381,괘달머리-적다,형용사,1,‘괘다리적다’를 속되게 이르는 말.,,,,,괘다리적다


##### 6. 본용언 + 연결어미 + 보조용언 -> 본용언을 추출
버리다, 놓다 등 마지막 보조 용언 리스트를 하은 님이 주시면 앞 바로 앞 어절이 ~을, ~를로 끝나지 않는 것만 추출

In [158]:
# <보조용언 리스트>
verb_list = ['가다', '오다', '두다', '버리다', '주다', '보다', '놓다', '싶다']

In [173]:
def extract6(x):
  temp = x.split('.')
  temp = [i+'.' for i in temp if i != '']

  result = []
  for i in temp:
    for v in verb_list:
      m = re.search(f'[^을를$] +{v}.$', i.strip())
      if m:
        result.append(m.group().split()[-1].replace('.', ''))
      
  return ', '.join(result)

In [176]:
df['extract6'] = df['mean'].swifter.apply(extract6)

Pandas Apply:   0%|          | 0/88508 [00:00<?, ?it/s]

In [180]:
check = df[df['mean'].str.contains("버리다.")]

In [182]:
check.head()

,vocab,pos,mean_no,mean,extract1,extract2,extract3,extract4,extract5,extract6
490,가무-뜨리다,동사,1,아주 자취 없이 가무려 버리다.,버리다,,,,,버리다
495,가무리다,동사,1,몰래 혼자 차지하거나 흔적도 없이 먹어 버리다.,버리다,,,,,버리다
506,가무-트리다,동사,1,아주 자취 없이 가무려 버리다.,버리다,,,,,버리다
682,가위질-하다,동사,2,언론 기사나 영화 작품 따위를 미리 검사하여 그 일부분을 지워 버리다.,버리다,,,,,버리다
962,각산진비-하다,동사,1,저마다 따로 흩어져 모두 가 버리다.,버리다,,,,,버리다


In [183]:
df.head()

,vocab,pos,mean_no,mean,extract1,extract2,extract3,extract4,extract5,extract6
0,가가대소-하다,동사,1,소리 내어 크게 웃다.,웃다,크게 웃다,,,,
1,가가-하다,형용사,1,잘되었거나 착하다고 여길 만하다.,만하다,,,,,
2,가각-하다,형용사,1,인정이 없고 모질다.,모질다,,,없고 모질다,,
3,가감-되다,동사,1,더해지거나 빼지다. 또는 더해지거나 빼져서 알맞게 맞추어지다.,"빼지다, 맞추어지다",알맞게 맞추어지다,더해지거나 빼지다,,,
4,가감-하다,동사,1,더하거나 빼다. 또는 더하거나 빼서 알맞게 맞추다.,"빼다, 맞추다",알맞게 맞추다,더하거나 빼다,,,
